In [1]:
import pandas as pd
import subprocess
import os
import numpy as np
from Bio import SeqIO, pairwise2
from Bio.PDB.DSSP import DSSP
from Bio.PDB import PDBParser, Superimposer
import warnings
import matplotlib.pyplot as plt
import json
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
warnings.filterwarnings('ignore')

/scicore/home/zavolan/zhu0006/.local/lib/python3.12/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


# Functions

In [2]:
#### RMSD ####

    
# to calculate rmsd using Bio.PDB
# predicted_pdb: moving pdb comparing to reference, need to have the same number of CA atoms as the ref_pdb
def rmsd_pdb(predicted_pdb, ref_pdb):
    parser = PDBParser()
    struct_ref = parser.get_structure(os.path.basename(ref_pdb), ref_pdb)
    struct_predicted = parser.get_structure(os.path.basename(predicted_pdb), predicted_pdb)
    fixed = [atom for atom in struct_ref[0].get_atoms() if atom.name == "CA"]
    moving = [atom for atom in struct_predicted[0].get_atoms() if atom.name == "CA"]
    sup = Superimposer()
    # sets the fixed and moving atom lists
    # finds the rotation and translation matrix that best superimposes the moving atoms onto fixed atoms
    sup.set_atoms(fixed, moving)
    # applies the calculated rotation and translation to all atoms in the second structure
    # superimposing it onto the first structure
    #sup.apply(struct_predicted[0].get_atoms())
    sup.apply(moving)

    return sup.rms


def rmsd_point(coordinate1, coordinate2):
    # 3D coordinates, example: array([ 11.925492,  10.070204, -12.518902], dtype=float32)
    # this is a function to calculate rmsd for a single point
    x1 = coordinate1[0]
    y1 = coordinate1[1]
    z1 = coordinate1[2]
    x2 = coordinate2[0]
    y2 = coordinate2[1]
    z2 = coordinate2[2]
    value = np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)
    return value


def rmsd_list(coordinates1, coordinates2):
    # list of 3D coordinates, should have the same number of coordinates
    # this is a function to calculate rmsd for two list of 3D coordinates
    length = len(coordinates1)
    values = []
    for i in range(length):
        x1 = coordinates1[i][0]
        y1 = coordinates1[i][1]
        z1 = coordinates1[i][2]
        x2 = coordinates2[i][0]
        y2 = coordinates2[i][1]
        z2 = coordinates2[i][2]
        value = (x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2
        values.append(value)
    rmsd = np.sqrt(sum(values)/length)
    return rmsd



#### Pairwise Sequence Alignment ####
    

# load fasta file to get the sequence
def load_fasta(fasta_file):
    with open(fasta_file, 'r') as f:
        record = next(SeqIO.parse(f, 'fasta'))
        return record.seq


# extract common residues from two sequences
def common_residues(seq1, seq2):
    # inputs are aligned sequences, including gaps, length should be the same
    # outputs are the common residues of the two sequences, indices are the original ones (without gaps)
    common_indices1 = []
    common_indices2 = []
    res_idx1 = 0
    res_idx2 = 0
    for a, b in zip(seq1, seq2):
        if a == b:
            res_idx1 += 1
            res_idx2 += 1
            common_indices1.append(res_idx1)
            common_indices2.append(res_idx2)
        elif (a != b) and (a != '-') and (b != '-'):
            res_idx1 += 1
            res_idx2 += 1
        elif (a == '-') and (b != '-'):
            res_idx2 += 1
        elif (a != '-') and (b == '-'):
            res_idx1 += 1
        else:
            print(a, b)
    if len(common_indices1) != len(common_indices2):
        print("Two indices have different length!")
        return 1
    return common_indices1, common_indices2

    
# return common indices for each sequence
def pairwise_SA(moving_fasta, ref_fasta):
    # load sequences from fasta files
    seq_moving = load_fasta(moving_fasta)
    seq_ref = load_fasta(ref_fasta)

    # pairwise sequence alignment
    alignments = pairwise2.align.globalxx(seq_moving, seq_ref)
    aligned_seq_moving, aligned_seq_ref = alignments[0][:2]

    # extract common residues
    indices_moving, indices_ref = common_residues(aligned_seq_moving, aligned_seq_ref)

    return indices_moving, indices_ref


#### read residue types from pairs ####
def residue_type(moving_fasta, ref_fasta):
    # load sequences from fasta files
    seq_moving = load_fasta(moving_fasta)
    seq_ref = load_fasta(ref_fasta)

    # pairwise sequence alignment
    alignments = pairwise2.align.globalxx(seq_moving, seq_ref)
    aligned_seq_moving, aligned_seq_ref = alignments[0][:2]

    # extract common residues
    indices_moving, indices_ref = common_residues(aligned_seq_moving, aligned_seq_ref)

    # initialize dictionary to save residue types
    residue_moving = {}
    residue_ref = {}

    # save the residue
    for i in range(len(indices_ref)):
        idx_moving = indices_moving[i] 
        idx_ref = indices_ref[i]
        residue_moving[idx_ref] = seq_moving[idx_moving-1] # use reference index
        residue_ref[idx_ref] = seq_ref[idx_ref-1] # change 1-based index to 0-based index

    # check if the two dictionaries are the same
    if residue_moving != residue_ref:
        print("residue inconsistency!")
        return 1

    # only need to return one dictionary
    return residue_ref
    
    
#### RMSD with pair ####
    
# to calculate rmsd per residue using Bio.PDB
def rmsd_pdb_perResidue(moving_pdb, ref_pdb, moving_fasta, ref_fasta):
    # load PDB structures
    parser = PDBParser()
    struct_moving = parser.get_structure(os.path.basename(moving_pdb), moving_pdb)
    struct_ref = parser.get_structure(os.path.basename(ref_pdb), ref_pdb)

    # extract common residues
    indices_moving, indices_ref = pairwise_SA(moving_fasta, ref_fasta)
    
    # get CA atoms from PDBs
    moving = [atom for atom in struct_moving[0].get_atoms() if (atom.name == "CA") and (atom.full_id[3][1] in indices_moving)]
    fixed = [atom for atom in struct_ref[0].get_atoms() if (atom.name == "CA") and (atom.full_id[3][1] in indices_ref)]

    # check if the two structures have the same number of length
    if len(moving) != len(fixed):
        print("Two structures have different numbers of residues!")
    
    # get the fixed coordinates
    coords_fixed = []
    for i in range(len(fixed)):
        coords_fixed.append(fixed[i].get_coord())
    # get the moving coordinates
    sup = Superimposer()
    sup.set_atoms(fixed, moving)
    sup.apply(moving)
    coords_moving = []
    for i in range(len(moving)):
        coords_moving.append(moving[i].get_coord())

    # calculate rmsd per residue (CA)
    rmsd_perResidue = {}
    for i in range(len(coords_fixed)):
        residue_id = fixed[i].full_id[3][1]
        rmsd_perResidue[residue_id] = rmsd_point(coords_fixed[i], coords_moving[i])

    return rmsd_perResidue


# to calculate rmsd overall using Bio.PDB
# return rmsd between two structures
# indices consistent with the previous function "rmsd_pdb_perResidue"
def rmsd_pdb_overall(moving_pdb, ref_pdb, moving_fasta, ref_fasta):
    # load PDB structures
    parser = PDBParser()
    struct_moving = parser.get_structure(os.path.basename(moving_pdb), moving_pdb)
    struct_ref = parser.get_structure(os.path.basename(ref_pdb), ref_pdb)

    # extract common residues
    indices_moving, indices_ref = pairwise_SA(moving_fasta, ref_fasta)
    
    # get CA atoms from PDBs
    moving = [atom for atom in struct_moving[0].get_atoms() if (atom.name == "CA") and (atom.full_id[3][1] in indices_moving)]
    fixed = [atom for atom in struct_ref[0].get_atoms() if (atom.name == "CA") and (atom.full_id[3][1] in indices_ref)]

    # check if the two structures have the same number of length
    if len(moving) != len(fixed):
        print("Two structures have different numbers of residues!")
    
    # get the fixed coordinates
    coords_fixed = []
    for i in range(len(fixed)):
        coords_fixed.append(fixed[i].get_coord())
    # get the moving coordinates
    sup = Superimposer()
    sup.set_atoms(fixed, moving)
    sup.apply(moving)
    coords_moving = []
    for i in range(len(moving)):
        coords_moving.append(moving[i].get_coord())

    rmsd = rmsd_list(coords_moving, coords_fixed)

    return rmsd



#### delta delta G ####
    
# extract delta G per residue out of sc file
def dG_perResidue(sc_path):
    dG_perResidue = {}
    with open(sc_path, 'r') as f:
        for count, line in enumerate(f.readlines()):
            if(count != 0):
                line = line.strip("\n").split()
                id = int(line[23].split("_")[1])
                score = float(line[22])
                dG_perResidue[id] = score
    return dG_perResidue

# to calculate ddG within pairs
def ddG_perResidue(moving_sc, ref_sc, moving_fasta, ref_fasta):
    #### input csv files !!! ####

    # extract common residues
    indices_moving, indices_ref = pairwise_SA(moving_fasta, ref_fasta)

    # read score file
    dG_moving = pd.read_csv(moving_sc)
    dG_ref = pd.read_csv(ref_sc)

    # initiate dG and ddG dictionary
    dG_iso_perResidue = {}
    dG_ref_perResidue = {}
    ddG_perResidue = {}
    
    # dG and ddG
    for i in range(len(indices_ref)):
        idx_moving = indices_moving[i]
        score_moving = dG_moving[dG_moving['residue_id']==idx_moving]['dG'].iloc[0]
        idx_ref = indices_ref[i]
        score_ref = dG_ref[dG_ref['residue_id']==idx_ref]['dG'].iloc[0]
        ddG = round(score_moving - score_ref, 3)

        # use index for reference as residue id
        dG_iso_perResidue[idx_ref] = score_moving
        dG_ref_perResidue[idx_ref] = score_ref
        ddG_perResidue[idx_ref] = ddG

    return ddG_perResidue, dG_iso_perResidue, dG_ref_perResidue
    

#### extract fa_sol term out of sc files ####
def fa_sol_perResidue(sc_path):
    fa_sol_perResidue = {}
    with open(sc_path, 'r') as f:
        for count, line in enumerate(f.readlines()):
            if (count != 0):
                line = line.strip("\n").split()
                id = int(line[23].split("_")[1])
                fa_sol = float(line[5])
                fa_sol_perResidue[id] = fa_sol
    return fa_sol_perResidue

    
# get fa_sol within pairs
def get_pair_fa_sol(moving_sc, ref_sc, moving_fasta, ref_fasta):
    #### input csv files !!! ####

    # extract common residues
    indices_moving, indices_ref = pairwise_SA(moving_fasta, ref_fasta)

    # read score file
    fa_sol_moving = pd.read_csv(moving_sc)
    fa_sol_ref = pd.read_csv(ref_sc)

    # initiate fa_sol dictionary
    fa_sol_iso_perResidue = {}
    fa_sol_ref_perResidue = {}

    # fa_sol
    for i in range(len(indices_ref)):
        idx_moving = indices_moving[i]
        score_moving = fa_sol_moving[fa_sol_moving['residue_id']==idx_moving]['fa_sol'].iloc[0]
        idx_ref = indices_ref[i]
        score_ref = fa_sol_ref[fa_sol_ref['residue_id']==idx_ref]['fa_sol'].iloc[0]

        # use index for reference as residue id
        fa_sol_iso_perResidue[idx_ref] = score_moving
        fa_sol_ref_perResidue[idx_ref] = score_ref

    return fa_sol_iso_perResidue, fa_sol_ref_perResidue

    
# get tags with the lowest relax score
# here it generates a tag file (basically a txt file)
# that contains 3 ids with lowest score by default

def get_lowestTag(sc_file, tag_file, num = 3):
    if sc_file==tag_file:
        print("score file and tag file should be different!")
        return 1
    # sc file is the one generated by Rosetta Relax
    # tag file is a path, not a folder, need to specify the file name
    scores_and_ids = pd.DataFrame(columns = ['score', 'id'])
    with open(sc_file, "r") as f:
        for count, line in enumerate(f.readlines()):
            if (count != 0) and (count != 1): # excluded the first two lines
                line = line.strip("\n")
                line = line.split()
                scores_and_ids.loc[len(scores_and_ids)] = [float(line[1]), line[23]]
    scores_and_ids = scores_and_ids.sort_values(by = 'score', ascending = True)
    lowest_ids = scores_and_ids['id'].head(num)
    with open(tag_file, 'w') as f:
        for id in lowest_ids:
            f.write(f"{id}\n")
    return 0


# get the lowest relax score
def get_lowestScore(sc_file):
    # very similar to the function get_lowestTag
    # sc file is the one generated by Rosetta Relax
    scores_and_ids = pd.DataFrame(columns = ['score', 'id'])
    with open(sc_file, 'r') as f:
        for count, line in enumerate(f.readlines()):
            if (count != 0) and (count != 1): # excluded the first two lines
                line = line.strip("\n")
                line = line.split()
                scores_and_ids.loc[len(scores_and_ids)] = [float(line[1]), line[23]]
    scores_and_ids = scores_and_ids.sort_values(by = 'score', ascending = True)
    lowest_score = float(scores_and_ids['score'].head(1))
    lowest_tag = scores_and_ids['id'].head(1).iloc[0]
    return lowest_score, lowest_tag
    

#### plddt score (with pair) ####
    
# get plddt score from "ranking_debug.json"
def get_plddt(af_ranking_file):
    af_ranking = json.load(open(af_ranking_file))
    ave_plddt = format(sum(af_ranking['plddts'].values()) / len(af_ranking['plddts'].values()), '.3f')
    return float(ave_plddt)


# get lowest plddt score from "ranking_debug.json"
def get_plddt_highest(af_ranking_file):
    af_ranking = json.load(open(af_ranking_file))
    plddt = max(af_ranking['plddts'].values())
    return plddt

# get plddt per residue
# based on the original residue id within the corresponding pdb file
def get_plddt_perResidue(result_model_pkl):
    plddt_dic = {} # create a dictionary to store the plddt score
    plddt = pd.read_pickle(result_model_pkl)
    plddt = plddt['plddt']
    for i in range(len(plddt)):
        plddt_dic[i+1] = plddt[i]
    return plddt_dic


# get model name for highest ranking
def get_ranked_0_model(af_ranking_file):
    af_ranking = json.load(open(af_ranking_file))
    model_name = af_ranking['order'][0]
    return model_name


# get plddt per residue, index based on pairwise sequence alignment
def plddt_pair_perResidue(moving_pkl, ref_pkl, moving_fasta, ref_fasta):
    ### input pkl files

    # extract common residues
    indices_moving, indices_ref = pairwise_SA(moving_fasta, ref_fasta)

    # read pkl files
    moving_plddt = get_plddt_perResidue(moving_pkl)
    ref_plddt = get_plddt_perResidue(ref_pkl)

    # initiate plddt dictionary
    moving_plddt_perResidue = {}
    ref_plddt_perResidue = {}
    
    # change the index based on sequence alignment
    for i in range(len(indices_ref)):
        idx_moving = indices_moving[i]
        idx_ref = indices_ref[i]
        plddt_score_moving = moving_plddt[idx_moving]
        plddt_score_ref = ref_plddt[idx_ref] # get plddt score from dictionary
        moving_plddt_perResidue[idx_ref] = plddt_score_moving # use reference index for consistency
        ref_plddt_perResidue[idx_ref] = plddt_score_ref

    return moving_plddt_perResidue, ref_plddt_perResidue




    
#### RSA from DSSP ####


# get relative accessible surface
def get_dssp(pdb_file):
    # need to import #
    # from Bio.PDB.DSSP import DSSP
    # from Bio.PDB import PDBParser
    p = PDBParser()
    structure = p.get_structure(str(pdb_file), pdb_file)
    model = structure[0]

    # count residues
    #n = 0
    #for res in model.get_residues():
        #n += 1
    
    dssp = DSSP(model, pdb_file, dssp = 'mkdssp')
    #if len(dssp.keys()) != n:
        #print(os.path.basename(pdb_file) + " dssp length different from pdb!")
        #return 1
    return dssp


# change from residue index to dssp index
def fix_index(pdb_file, chain_id, residue_id):
    # from Bio.PDB.DSSP import dssp_dict_from_pdb_file

    # define chain
    #chain_id = os.path.basename(pdb_file).split('.')[0].split('_')[1]

    # get DSSP index using residue index
    dssp_tuple = dssp_dict_from_pdb_file(pdb_file)
    dssp_dict = dssp_tuple[0]
    key = (chain_id, (' ', residue_id, ' '))
    if key in dssp_dict.keys():
        dssp_id = dssp_dict[chain_id, (' ', residue_id, ' ')][5]
        # reference: https://github.com/biopython/biopython/blob/master/Bio/PDB/DSSP.py
        return dssp_id
    return None

    
# get single RSA value given the residue ID
def get_single_rsa(pdb_file, chain_id, residue_id):
    dssp_data = get_dssp(pdb_file)
    dssp_id = fix_index(pdb_file, chain_id, residue_id)

    if dssp_id == None:
        return None

    for key in dssp_data.keys():
        if dssp_data[key][0] == dssp_id:
            # dssp_data[key][0] is DSSP index
            # according to https://biopython.org/docs/1.76/api/Bio.PDB.DSSP.html
            return dssp_data[key][3]
    return None

    
# get RSA per residue, index based on pairwise sequence alignment
def get_pair_rsa(moving_pdb, ref_pdb, moving_fasta, ref_fasta, moving_chain_id, ref_chain_id):

    # need to import #
    # from Bio import SeqIO, pairwise2
    # from Bio.PDB.DSSP import DSSP
    # from Bio.PDB import PDBParser

    # extract common residues
    indices_moving, indices_ref = pairwise_SA(moving_fasta, ref_fasta)

    # get dssp
    #moving_dssp = get_dssp(moving_pdb)
    #ref_dssp = get_dssp(ref_pdb)

    # initiate rsa dictionary
    moving_rsa_perResidue = {}
    ref_rsa_perResidue = {}

    # index based on reference sequence
    for i in range(len(indices_ref)):
        idx_moving = indices_moving[i]
        idx_ref = indices_ref[i]
        
        # get RSA
        rsa_moving = get_single_rsa(moving_pdb, moving_chain_id, idx_moving)
        if rsa_moving != None:
            moving_rsa_perResidue[idx_ref] = rsa_moving # save in dictionary, use reference index for consistency
            
        rsa_ref = get_single_rsa(ref_pdb, ref_chain_id, idx_ref)
        if rsa_ref != None:
            ref_rsa_perResidue[idx_ref] = rsa_ref

    return moving_rsa_perResidue, ref_rsa_perResidue

# Read xlsx table

In [7]:
support_lv = "No_Evidence"
df = pd.read_excel("Readthrough_Sequences.xlsx", sheet_name = support_lv)
if support_lv == "No_Evidence":
    df = df.sample(n=1000, random_state=42)


# Proteomics_Evidence
#colname_C = "Canonical"
#colname_R = "Readthrough"
# RPF_Evidence & No_Evidence
colname_C = "Main CDS"
colname_R = "Readthrough form"

# Preprocessing

In [6]:
sc_path = os.path.join("structure_res", "No_Evidence", "P11532_18319_R", "relax_lowestScore_perRes.sc")
sc = dG_perResidue(sc_path)
sc = pd.DataFrame.from_dict(sc, orient = 'index', columns = ['dG'])
sc = sc.reset_index().rename(columns = {'index': 'residue_id'})
csv_path = os.path.join("structure_res", "No_Evidence", "P11532_18319_R", "relax_lowestScore_perRes.csv")
sc.to_csv(csv_path, index = False)

In [4]:
# extract delta G from sc files, and save in csv files

alphafold_res_path = os.path.join("structure_res", support_lv)

for protein_id in os.listdir(alphafold_res_path):
    # sc file path
    sc_path = os.path.join(alphafold_res_path, protein_id, "relax_lowestScore_perRes.sc")
    # check if sc file exists
    if not os.path.exists(sc_path): # if sc file does not exist
        print(protein_id) # print the protein id
        continue
    # read sc file
    sc = dG_perResidue(sc_path)
    sc = pd.DataFrame.from_dict(sc, orient = 'index', columns = ['dG'])
    sc = sc.reset_index().rename(columns = {'index': 'residue_id'})
    # save csv file
    csv_path = os.path.join(alphafold_res_path, protein_id, "relax_lowestScore_perRes.csv")
    sc.to_csv(csv_path, index = False)

print("Done!")

P17612_2874_R
Q96RU7_9682_C
Q96RU7_9682_R
P55073_14410_R
Q2WEN9_3002_R
P55073_14410_C
Q86YV6_10903_R
Q9H609_9176_R
P24723_14339_C
Q6V0I7_4407_R
P11532_18319_R
Q8N2I9_6366_R
Q93033_6549_R
Q9UPN3_12589_R
P24723_14339_R
Q9Y616_14021_R
Q5H9U9_16912_C
P24043_11216_R
Q8N2I9_6366_C
Q2WEN9_3002_C
P52739_10662_C
Q86YV6_10903_C
Q6V0I7_4407_C
Q2M2H8_11519_R
Q8NCM8_13745_R
P52739_10662_R
Q9H609_9176_C
Q8NCM8_13745_C
Q9Y616_14021_C
Q9UPN3_12589_C
P17612_2874_C
Q93033_6549_C
Done!


# Calculate metrics

In [ ]:
####### DO NOT RUN!! #######
####### Submit by Slurm #######

# Loop through protein ids

alphafold_res_path = os.path.join("structure_res", support_lv)
tag_filename = "lowest.tag"
sc_csv_filename = "relax_lowestScore_perRes.csv"

for index, row in df.iterrows():
    # get ids
    id_C = row['Protein'] + "_" + str(index) + "_C"
    id_R = row['Protein'] + "_" + str(index) + "_R"

    # alphafold results path
    res_path_C = os.path.join(alphafold_res_path, id_C)
    res_path_R = os.path.join(alphafold_res_path, id_R)

    # fasta paths
    fasta_name_C = id_C + ".fasta"
    fasta_path_C = os.path.join("fasta", support_lv, fasta_name_C)
    fasta_name_R = id_R + ".fasta"
    fasta_path_R = os.path.join("fasta", support_lv, fasta_name_R)

    # tag paths (with lowest relax score)
    tag_path_C = os.path.join(res_path_C, tag_filename)
    tag_path_R = os.path.join(res_path_R, tag_filename)

    # structure basenames (with lowest relax score)
    with open(tag_path_C, 'r') as f:
        relax_basename_C = f.readline().strip()
    with open(tag_path_R, 'r') as f:
        relax_basename_R = f.readline().strip()

    # pdb file paths (with lowest relax score)
    pdb_name_C = relax_basename_C + ".pdb"
    pdb_path_C = os.path.join(res_path_C, pdb_name_C)
    pdb_name_R = relax_basename_R + ".pdb"
    pdb_path_R = os.path.join(res_path_R, pdb_name_R)

    # score per residue csv file paths
    sc_path_C = os.path.join(res_path_C, sc_csv_filename)
    sc_path_R = os.path.join(res_path_R, sc_csv_filename)


    #### indices for isoform ####
    indices_R, indices_C = pairwise_SA(moving_fasta=fasta_path_R, ref_fasta=fasta_path_C) # return moving indices first (readthrough), then reference indices (canonical)
    indices_dict = dict(zip(indices_C, indices_R)) # use canonical indices as keys
    # initialize the dataframe
    pair_df = pd.DataFrame({
        'Residue_C': indices_dict.keys(),
        'Residue_R': indices_dict.values()
    })


    #### add residue types ####
    residues = residue_type(moving_fasta=fasta_path_R, ref_fasta=fasta_path_C)
    # combine into the dataframe
    pair_df['ResidueType'] = pair_df['Residue_C'].map(residues)


    #### RMSD ####
    rmsd = rmsd_pdb_perResidue(moving_pdb=pdb_path_R, ref_pdb=pdb_path_C, moving_fasta=fasta_path_R, ref_fasta=fasta_path_C)
    # combine into the dataframe
    pair_df['RMSD'] = pair_df['Residue_C'].map(rmsd)

    
    #### dG and ddG ####
    ddG, dG_R, dG_C = ddG_perResidue(moving_sc=sc_path_R, ref_sc=sc_path_C, moving_fasta=fasta_path_R, ref_fasta=fasta_path_C) #ddG=moving-ref
    # combine into the dataframe
    pair_df['ddG'] = pair_df['Residue_C'].map(ddG)
    pair_df['dG_R'] = pair_df['Residue_C'].map(dG_R)
    pair_df['dG_C'] = pair_df['Residue_C'].map(dG_C)


    #### plddt per residue ####
    model_name_C = get_ranked_0_model(af_ranking_file=os.path.join(res_path_C, "ranking_debug.json"))
    pkl_name_C = "result_" + model_name_C + ".pkl"
    pkl_path_C = os.path.join(res_path_C, pkl_name_C)
    model_name_R = get_ranked_0_model(af_ranking_file=os.path.join(res_path_R, "ranking_debug.json"))
    pkl_name_R = "result_" + model_name_R + ".pkl"
    pkl_path_R = os.path.join(res_path_R, pkl_name_R)
    plddt_R, plddt_C = plddt_pair_perResidue(moving_pkl=pkl_path_R, ref_pkl=pkl_path_C, moving_fasta=fasta_path_R, ref_fasta=fasta_path_C)
    # combine into the dataframe
    pair_df['plddt_R'] = pair_df['Residue_C'].map(plddt_R)
    pair_df['plddt_C'] = pair_df['Residue_C'].map(plddt_C)


    #### RSA per residue ####
    rsa_R, rsa_C = get_pair_rsa(moving_pdb=pdb_path_R, ref_pdb=pdb_path_C,
                                moving_fasta=fasta_path_R, ref_fasta=fasta_path_C,
                                moving_chain_id="A", ref_chain_id="A")
    # combine into the dataframe
    pair_df['rsa_R'] = pair_df['Residue_C'].map(rsa_R)
    pair_df['rsa_C'] = pair_df['Residue_C'].map(rsa_C)

    # print process
    print(f"Processed {id_C} and {id_R}")

    # save to csv
    csv_name = row['Protein'] + "_" + str(index) + ".csv"
    csv_path = os.path.join("pairs_csv", support_lv, csv_name)
    

In [8]:
#### metrics for pairs ####

# define hydrophilic & hydrophobic
hydrophobic = ['W', 'Y', 'F', 'I', 'L', 'M', 'V', 'A', 'C']
neutral = ['T', 'H', 'G', 'S', 'Q']
hydrophilic = ['R', 'E', 'N', 'K', 'P', 'D']

# protein id to skip
# Proteomics_Evidence
#skip_pid = []
# RPF_Evidence
#skip_pid = ['APOB_326', 'WDR36_361', 'NRBP2_466', 'ABHD4_557', 'THBS1_668', 'ALDH6A1_793', 'FKBP5_844', 'PIGR_889']
# No_Evidence
skip_pid = ['P17612_2874', 'Q96RU7_9682', 'P55073_14410', 'Q2WEN9_3002', 'Q86YV6_10903', 'Q9H609_9176', 'P24723_14339', 'Q6V0I7_4407', 'Q8N2I9_6366', 'Q93033_6549', 'Q9UPN3_12589', 'Q9Y616_14021', 'P52739_10662', 'Q8NCM8_13745']

# save everything in a list
rows = []


# loop through pairs
for index, row in df.iterrows():
    # protein id
    pid = row['Protein'] + "_" + str(index)

    # skip if pid is in the list
    if pid in skip_pid:
        continue

    # get ids for each sequence
    id_R = pid + "_R" # R for Readthrough
    id_C = pid + "_C" # C for Canonical

    # define relax sc path
    # this is the overall delta G for every single structure generated by RosettaRelax
    relax_sc_R = os.path.join("structure_res", support_lv, id_R, "relax.sc")
    relax_sc_C = os.path.join("structure_res", support_lv, id_C, "relax.sc")

    # define the csv path for current pair
    csv_path = os.path.join("pairs_csv", support_lv, pid+".csv")
    # read csv files
    pair_df = pd.read_csv(csv_path)

    # overall delta G values for both structures
    # get_lowestScore(sc_path)
    dG_R, tag_R = get_lowestScore(relax_sc_R)
    dG_C, tag_C = get_lowestScore(relax_sc_C)
    # 10% quantile for the lowest per-residue delta G values
    dG_10q_low_R = pair_df['dG_R'].quantile(0.1)
    dG_10q_high_R = pair_df['dG_R'].quantile(0.9)
    dG_10q_low_C = pair_df['dG_C'].quantile(0.1)
    dG_10q_high_C = pair_df['dG_C'].quantile(0.9)
    # 10% quantile for the per-residue ddG
    ddG_10q_low = pair_df['ddG'].quantile(0.1)
    ddG_10q_high = pair_df['ddG'].quantile(0.9)

    # overall plddt scores for both structures
    # define "ranking_debug.json" path
    json_R = os.path.join("structure_res", support_lv, id_R, "ranking_debug.json")
    json_C = os.path.join("structure_res", support_lv, id_C, "ranking_debug.json")
    # highest overall plddt score
    plddt_R = get_plddt_highest(json_R)
    plddt_C = get_plddt_highest(json_C)
    # 10% quantile for the lowest per-residue plddt scores
    plddt_10q_low_R = pair_df['plddt_R'].quantile(0.1)
    plddt_10q_high_R = pair_df['plddt_R'].quantile(0.9)
    plddt_10q_low_C = pair_df['plddt_C'].quantile(0.1)
    plddt_10q_high_C = pair_df['plddt_C'].quantile(0.9)

    # median RSA for both structures
    rsa_R = pair_df['rsa_R'].dropna().median()
    rsa_C = pair_df['rsa_C'].dropna().median()
    # median RSA of hydrophilic residues for both structures
    rsa_hydrophilic_R = pair_df[pair_df['ResidueType'].isin(hydrophilic)]['rsa_R'].dropna().median()
    rsa_hydrophilic_C = pair_df[pair_df['ResidueType'].isin(hydrophilic)]['rsa_C'].dropna().median()
    # median RSA of hydrophobic residues for both structures
    rsa_hydrophobic_R = pair_df[pair_df['ResidueType'].isin(hydrophobic)]['rsa_R'].dropna().median()
    rsa_hydrophobic_C = pair_df[pair_df['ResidueType'].isin(hydrophobic)]['rsa_C'].dropna().median()
    # median RSA of each residue type for both structures
    rsa_W_R = pair_df[pair_df['ResidueType']=='W']['rsa_R'].dropna().median()
    rsa_W_C = pair_df[pair_df['ResidueType']=='W']['rsa_C'].dropna().median() # Tryptophan
    rsa_Y_R = pair_df[pair_df['ResidueType']=='Y']['rsa_R'].dropna().median()
    rsa_Y_C = pair_df[pair_df['ResidueType']=='Y']['rsa_C'].dropna().median() # Tyrosine
    rsa_F_R = pair_df[pair_df['ResidueType']=='F']['rsa_R'].dropna().median()
    rsa_F_C = pair_df[pair_df['ResidueType']=='F']['rsa_C'].dropna().median() # Phenylalanine
    rsa_I_R = pair_df[pair_df['ResidueType']=='I']['rsa_R'].dropna().median()
    rsa_I_C = pair_df[pair_df['ResidueType']=='I']['rsa_C'].dropna().median() # Isoleucine
    rsa_L_R = pair_df[pair_df['ResidueType']=='L']['rsa_R'].dropna().median()
    rsa_L_C = pair_df[pair_df['ResidueType']=='L']['rsa_C'].dropna().median() # Leucine
    rsa_M_R = pair_df[pair_df['ResidueType']=='M']['rsa_R'].dropna().median()
    rsa_M_C = pair_df[pair_df['ResidueType']=='M']['rsa_C'].dropna().median() # Methionine
    rsa_V_R = pair_df[pair_df['ResidueType']=='V']['rsa_R'].dropna().median()
    rsa_V_C = pair_df[pair_df['ResidueType']=='V']['rsa_C'].dropna().median() # Valine
    rsa_A_R = pair_df[pair_df['ResidueType']=='A']['rsa_R'].dropna().median()
    rsa_A_C = pair_df[pair_df['ResidueType']=='A']['rsa_C'].dropna().median() # Alanine
    rsa_C_R = pair_df[pair_df['ResidueType']=='C']['rsa_R'].dropna().median()
    rsa_C_C = pair_df[pair_df['ResidueType']=='C']['rsa_C'].dropna().median() # Cysteine
    rsa_T_R = pair_df[pair_df['ResidueType']=='T']['rsa_R'].dropna().median()
    rsa_T_C = pair_df[pair_df['ResidueType']=='T']['rsa_C'].dropna().median() # Threonine
    rsa_H_R = pair_df[pair_df['ResidueType']=='H']['rsa_R'].dropna().median()
    rsa_H_C = pair_df[pair_df['ResidueType']=='H']['rsa_C'].dropna().median() # Histidine
    rsa_G_R = pair_df[pair_df['ResidueType']=='G']['rsa_R'].dropna().median()
    rsa_G_C = pair_df[pair_df['ResidueType']=='G']['rsa_C'].dropna().median() # Glycine
    rsa_S_R = pair_df[pair_df['ResidueType']=='S']['rsa_R'].dropna().median()
    rsa_S_C = pair_df[pair_df['ResidueType']=='S']['rsa_C'].dropna().median() # Serine
    rsa_Q_R = pair_df[pair_df['ResidueType']=='Q']['rsa_R'].dropna().median()
    rsa_Q_C = pair_df[pair_df['ResidueType']=='Q']['rsa_C'].dropna().median() # Glutamine
    rsa_R_R = pair_df[pair_df['ResidueType']=='R']['rsa_R'].dropna().median()
    rsa_R_C = pair_df[pair_df['ResidueType']=='R']['rsa_C'].dropna().median() # Arginine
    rsa_E_R = pair_df[pair_df['ResidueType']=='E']['rsa_R'].dropna().median()
    rsa_E_C = pair_df[pair_df['ResidueType']=='E']['rsa_C'].dropna().median() # Glutamic acid
    rsa_N_R = pair_df[pair_df['ResidueType']=='N']['rsa_R'].dropna().median()
    rsa_N_C = pair_df[pair_df['ResidueType']=='N']['rsa_C'].dropna().median() # Asparagine
    rsa_K_R = pair_df[pair_df['ResidueType']=='K']['rsa_R'].dropna().median()
    rsa_K_C = pair_df[pair_df['ResidueType']=='K']['rsa_C'].dropna().median() # Lysine
    rsa_P_R = pair_df[pair_df['ResidueType']=='P']['rsa_R'].dropna().median()
    rsa_P_C = pair_df[pair_df['ResidueType']=='P']['rsa_C'].dropna().median() # Proline
    rsa_D_R = pair_df[pair_df['ResidueType']=='D']['rsa_R'].dropna().median()
    rsa_D_C = pair_df[pair_df['ResidueType']=='D']['rsa_C'].dropna().median() # Aspartic acid

    # pdb path for both iso and ref
    pdb_name_R = tag_R + ".pdb"
    pdb_path_R = os.path.join("structure_res", support_lv, id_R, pdb_name_R)
    pdb_name_C = tag_C + ".pdb"
    pdb_path_C = os.path.join("structure_res", support_lv, id_C, pdb_name_C)

    # fasta files for both iso and ref
    fasta_name_R = id_R + ".fasta"
    fasta_path_R = os.path.join("fasta", support_lv, fasta_name_R)
    fasta_name_C = id_C + ".fasta"
    fasta_path_C = os.path.join("fasta", support_lv, fasta_name_C)

    # overall RMSD
    # rmsd_pdb_overall(moving_pdb, ref_pdb, moving_fasta, ref_fasta)
    rmsd = rmsd_pdb_overall(pdb_path_R, pdb_path_C, fasta_path_R, fasta_path_C)


    #### integrate everything into a dictionary, and append into list
    row = {'protein_id': pid,
           'rmsd': rmsd,
           'dG_R': dG_R,
           'dG_C': dG_C,
           'dG_10q_low_R': dG_10q_low_R,
           'dG_10q_high_R': dG_10q_high_R,
           'dG_10q_low_C': dG_10q_low_C,
           'dG_10q_high_C': dG_10q_high_C,
           'ddG_10q_low': ddG_10q_low,
           'ddG_10q_high': ddG_10q_high,
           'plddt_R': plddt_R,
           'plddt_C': plddt_C,
           'plddt_10q_low_R': plddt_10q_low_R,
           'plddt_10q_high_R': plddt_10q_high_R,
           'plddt_10q_low_C': plddt_10q_low_C,
           'plddt_10q_high_C': plddt_10q_high_C,
           'rsa_R': rsa_R,
           'rsa_C': rsa_C,
           'rsa_hydrophilic_R': rsa_hydrophilic_R,
           'rsa_hydrophilic_C': rsa_hydrophilic_C,
           'rsa_hydrophobic_R': rsa_hydrophobic_R,
           'rsa_hydrophobic_C': rsa_hydrophobic_C,
           'rsa_W_R': rsa_W_R,
           'rsa_W_C': rsa_W_C,
           'rsa_Y_R': rsa_Y_R,
           'rsa_Y_C': rsa_Y_C,
           'rsa_F_R': rsa_F_R,
           'rsa_F_C': rsa_F_C,
           'rsa_I_R': rsa_I_R,
           'rsa_I_C': rsa_I_C,
           'rsa_L_R': rsa_L_R,
           'rsa_L_C': rsa_L_C,
           'rsa_M_R': rsa_M_R,
           'rsa_M_C': rsa_M_C,
           'rsa_V_R': rsa_V_R,
           'rsa_V_C': rsa_V_C,
           'rsa_A_R': rsa_A_R,
           'rsa_A_C': rsa_A_C,
           'rsa_C_R': rsa_C_R,
           'rsa_C_C': rsa_C_C,
           'rsa_T_R': rsa_T_R,
           'rsa_T_C': rsa_T_C,
           'rsa_H_R': rsa_H_R,
           'rsa_H_C': rsa_H_C,
           'rsa_G_R': rsa_G_R,
           'rsa_G_C': rsa_G_C,
           'rsa_S_R': rsa_S_R,
           'rsa_S_C': rsa_S_C,
           'rsa_Q_R': rsa_Q_R,
           'rsa_Q_C': rsa_Q_C,
           'rsa_R_R': rsa_R_R,
           'rsa_R_C': rsa_R_C,
           'rsa_E_R': rsa_E_R,
           'rsa_E_C': rsa_E_C,
           'rsa_N_R': rsa_N_R,
           'rsa_N_C': rsa_N_C,
           'rsa_K_R': rsa_K_R,
           'rsa_K_C': rsa_K_C,
           'rsa_P_R': rsa_P_R,
           'rsa_P_C': rsa_P_C,
           'rsa_D_R': rsa_D_R,
           'rsa_D_C': rsa_D_C}
    
    rows.append(row) # save in list


metrics_df = pd.DataFrame(rows)

metrics_path = os.path.join("metrics", support_lv+"_2.csv")
metrics_df.to_csv(metrics_path, index = False)

# Test

In [1]:
pwd

'/scicore/home/zavolan/zhu0006/3D_structure/structural_stability_of_novel_orfs_hcc'

In [4]:
get_lowestScore("structure_res/Proteomics_Evidence/A2M_20_C/relax.sc")

(-4561.61, 'ranked_0_0002')

In [5]:
af_ranking = json.load(open("structure_res/Proteomics_Evidence/HLA-B_7_R/ranking_debug.json"))
af_ranking

{'plddts': {'model_1_pred_0': 86.50774708954988,
  'model_2_pred_0': 85.38132406896463,
  'model_3_pred_0': 85.84049771495353,
  'model_4_pred_0': 84.31954199957784,
  'model_5_pred_0': 85.05695920317687},
 'order': ['model_1_pred_0',
  'model_3_pred_0',
  'model_2_pred_0',
  'model_5_pred_0',
  'model_4_pred_0']}

In [23]:
get_plddt_highest("alphafold_res/bad_10_ref/ranking_debug.json")

96.15198577210786

In [10]:
# rmsd_pdb_overall(moving_pdb, ref_pdb, moving_fasta, ref_fasta)
rmsd_pdb_overall("alphafold_res/good_0_iso/ranked_4_0001.pdb", "alphafold_res/good_0_ref/ranked_1_0001.pdb", "aa_seq/good_0_iso.fasta", "aa_seq/good_0_ref.fasta")

27.046520133671045

In [7]:
indices_moving, indices_ref = pairwise_SA("fasta/Proteomics_Evidence/PRRG2_15_R.fasta", "fasta/Proteomics_Evidence/PRRG2_15_C.fasta")
indices_dict = dict(zip(indices_ref, indices_moving))
print(indices_dict)

{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 121: 121, 122: 

In [15]:
pair_bad_10_df = pd.read_csv("pairs_csv/pair_bad_10.csv")
len(pair_bad_10_df['ResidueType'].unique())

16

In [4]:
# def get_pair_rsa(moving_pdb, ref_pdb, moving_fasta, ref_fasta, moving_chain_id, ref_chain_id)
# return moving_rsa_perResidue, ref_rsa_perResidue
test_moving_rsa, test_ref_rsa = get_pair_rsa("structure_res/Proteomics_Evidence/PRRG2_15_R/ranked_2_0003.pdb",
                                             "structure_res/Proteomics_Evidence/PRRG2_15_C/ranked_0_0002.pdb",
                                             "fasta/Proteomics_Evidence/PRRG2_15_R.fasta",
                                             "fasta/Proteomics_Evidence/PRRG2_15_C.fasta",
                                             "A", "A")

In [6]:
test_ref_rsa

{1: 1.0,
 2: 0.8669354838709677,
 3: 0.3333333333333333,
 4: 0.6956521739130435,
 5: 0.6397058823529411,
 6: 0.5538461538461539,
 7: 0.22560975609756098,
 8: 0.524390243902439,
 9: 0.7378048780487805,
 10: 0.4573170731707317,
 11: 0.5855855855855856,
 12: 0.5691489361702128,
 13: 0.4716981132075472,
 14: 0.4329268292682927,
 15: 0.2887323943661972,
 16: 0.6056338028169014,
 17: 0.725925925925926,
 18: 0.6036585365853658,
 19: 0.6012269938650306,
 20: 0.676056338028169,
 21: 0.25384615384615383,
 22: 0.4117647058823529,
 23: 0.38461538461538464,
 24: 0.4690721649484536,
 25: 0.8092783505154639,
 26: 0.9859154929577465,
 27: 0.50920245398773,
 28: 0.8080808080808081,
 29: 0.9381443298969072,
 30: 0.647887323943662,
 31: 0.4720812182741117,
 32: 0.3475609756097561,
 33: 0.20238095238095238,
 34: 0.5808823529411765,
 35: 0.7205882352941176,
 36: 0.5154639175257731,
 37: 0.018867924528301886,
 38: 0.601010101010101,
 39: 0.5307692307692308,
 40: 0.5634517766497462,
 41: 0.4329268292682927,


In [6]:
get_lowestScore("structure_res/Proteomics_Evidence/ARHGEF12_0_R/relax.sc")

(8627.18, 'ranked_1_0005')